In [2]:
import pandas as pd
import numpy as np

In [4]:
data1 = pd.read_excel(r"F:\temp\190529\数据准备\3&7日189.xlsx")
data2 = pd.read_excel(r"F:\temp\190529\数据准备\3&7日前新生产线下京东下省.xlsx")
data3 = pd.read_excel(r"F:\temp\190529\数据准备\首充.xlsx", sheet_name="Sheet1")
data4 = pd.read_excel(r"F:\temp\190529\数据准备\首充0530.xlsx")
data5 = pd.read_excel(r"F:\temp\190529\数据准备\业务订单明细[20190530164757].xlsx")


In [5]:
df1 = data1.copy()
df2 = data2.copy()
df3 = data3.copy()
df4 = data4.copy()
df5 = data5.copy()

In [6]:
def fahuo(a,b,c):
    if a is not np.nan:
        return 1
    elif a is np.nan and b == "交易完成":
        return 1
    elif a is np.nan and c is not np.nan:
        return 1
    else:
        return 0

In [7]:
def qianshou(a,b):
    if a is not np.nan:
        return 1
    elif a is np.nan and b == "交易完成":
        return 1
    else:
        return 0

In [8]:
def jihuo(x):
    if x == "交易完成":
        return 1
    else:
        return 0

In [9]:
def paidan(x):
    if x is np.nan:
        return 0
    else:
        return 1

In [10]:
df1 = df1[(True ^ df1['订单状态'].isin(['办理成功', '办理中', '办理失败']))]
df1 = df1[pd.notnull(df1['所在省 / 市 / 县'])]
df1 = df1[df1["订单编号"].str.contains('订单编号') == False]
split1 = pd.DataFrame(
    (x.split(' ') for x in df1['订单生成时间']),
    index=df1.index,
    columns=[
        '订单生成日期',
        '订单生成小时'])
df1 = pd.merge(df1, split1, left_index=True, right_index=True)

In [11]:
df2 = df2[df2["模式分类"] == "京东模式"]
df2 = df2[pd.notnull(df2['是否下省'])]

In [12]:
df34 = pd.concat([df3, df4])
df34 = df34.drop_duplicates(subset=['用户名'], keep='first')
df34 = df34[df34["订单编号"].str.contains('订单编号') == False]
df34['用户名'] = df34['用户名'].map(lambda x: round(x, 0))

In [13]:
df51 = df5[["运营商单号", "物流单号"]]
df52 = df5[["运营商单号", "APP操作时间"]]
df51 = df51[pd.notnull(df51['物流单号'])]
df52 = df52[pd.notnull(df52['APP操作时间'])]

In [14]:
df11 = pd.merge(df1, df2[["订单编号", "模式分类", "是否下省"]],
                left_on="订单编号", right_on="订单编号", how="left")
df11 = pd.merge(df11, df51, left_on="订单编号", right_on="运营商单号", how="left")
df11 = pd.merge(df11, df52, left_on="订单编号", right_on="运营商单号", how="left")
df11['入网手机号'] = df11['入网手机号'].apply(int)
df11 = pd.merge(df11, df34[['订单编号', '用户名']],
                left_on="入网手机号", right_on="用户名", how="left")


In [16]:
df11['派单'] = df11["是否下省"].apply(paidan)
df11['派卡'] = df11["物流单号_y"].apply(paidan)
df11['上门'] = df11["APP操作时间"].apply(paidan)
df11['首充'] = df11["订单编号_y"].apply(paidan)

In [22]:
print(df11.columns)

Index(['订单编号_x', '原始订单', '订单类型', '订单状态', '订单子状态', '取消原因', '订单挂起原因', '安装失败原因',
       '订单来源', '结算状态',
       ...
       '运营商单号_x', '物流单号_y', '运营商单号_y', 'APP操作时间', '订单编号_y', '用户名_y', '派单',
       '派卡', '上门', '首充'],
      dtype='object', length=124)


In [19]:
df11['发货量'] = df11.apply(lambda x: fahuo(x["物流单号_x"], x["订单状态"], x["物流签收时间"]), axis=1)

In [20]:
df11['签收量'] = df11.apply(lambda x: qianshou(x["物流签收时间"], x["订单状态"]), axis=1)

In [21]:
df11['激活量'] = df11["订单状态"].apply(jihuo)

In [23]:
df11.to_excel(r"F:\temp\190529\数据准备\test3.xlsx", index=False)
